# Задание 1

        Реализуйте программу на C++, которая динамически выделяет массив из 50 000 целых
        чисел, заполняет его случайными значениями от 1 до 100, вычисляет среднее значение
        элементов массива и корректно освобождает выделенную память.

In [17]:
%%writefile task1.cpp

#include <iostream>   // библиотека для ввода и вывода (cout, endl)
#include <cstdlib>    // библиотека для работы с rand() и srand()
#include <ctime>      // библиотека для получения текущего времени (time)

using namespace std;  // чтобы не писать std:: перед cout, endl 

int main() {
    // Константа, задающая размер массива
    const int N = 50000;

    // Динамическое выделение памяти под массив из N целых чисел
    int* arr = new int[N];

    // Инициализация генератора случайных чисел текущим временем
    // Это нужно, чтобы при каждом запуске числа были разными
    srand(time(nullptr));

    // Заполнение массива случайными числами в диапазоне от 1 до 100
    for (int i = 0; i < N; i++) {
        arr[i] = rand() % 100 + 1;
    }

    // Переменная для хранения суммы элементов массива
    // Используем тип long long, чтобы избежать переполнения
    long long sum = 0;

    // Последовательное суммирование всех элементов массива
    for (int i = 0; i < N; i++) {
        sum += arr[i];
    }

    // Вычисление среднего значения элементов массива
    // Приводим сумму к double для получения дробного результата
    double average = static_cast<double>(sum) / N;

    // Вывод среднего значения на экран
    cout << "Среднее значение: " << average << endl;

    // Освобождение ранее выделенной динамической памяти
    delete[] arr;

    // Завершение программы
    return 0;
}

Overwriting task1.cpp


In [18]:
!g++ task1.cpp -o task1
!./task1

Среднее значение: 50.3846


# Задание 2 

    Создайте массив из 1 000 000 целых чисел и реализуйте последовательный алгоритм
    поиска минимального и максимального элементов. Замерьте время выполнения алгоритма.

In [19]:
%%writefile task2.cpp

#include <iostream>   // ввод и вывод (cout, endl)
#include <cstdlib>    // rand(), srand()
#include <ctime>      // time() для инициализации генератора случайных чисел
#include <chrono>     // библиотека для измерения времени выполнения

using namespace std;       
using namespace chrono;     

int main() {
    const int N = 1000000;   // размер массива: 1 000 000 элементов

    // Динамическое выделение памяти под массив из N целых чисел
    int* arr = new int[N];

    // Инициализация генератора случайных чисел текущим временем
    srand(time(nullptr));

    // Заполнение массива случайными числами в диапазоне [1; 1 000 000]
    for (int i = 0; i < N; i++) {
        arr[i] = rand() % 1000000 + 1;
    }

    // Последовательный поиск min и max
    // -------------------------------

    // Фиксируем время начала выполнения алгоритма
    auto start = high_resolution_clock::now();

    // Инициализируем минимальное и максимальное значение первым элементом массива
    int min_val = arr[0];
    int max_val = arr[0];

    // Проходим по массиву и обновляем min и max
    for (int i = 1; i < N; i++) {
        if (arr[i] < min_val)
            min_val = arr[i];

        if (arr[i] > max_val)
            max_val = arr[i];
    }

    // Фиксируем время окончания выполнения алгоритма
    auto end = high_resolution_clock::now();

    // Вычисляем время выполнения в миллисекундах
    double time_ms = duration<double, milli>(end - start).count();

    // Вывод результатов на экран
    cout << "Минимальное значение: " << min_val << endl;
    cout << "Максимальное значение: " << max_val << endl;
    cout << "Время выполнения: " << time_ms << " мс" << endl;

    // Освобождаем динамически выделенную память
    delete[] arr;

    return 0;  // успешное завершение программы
}

Overwriting task2.cpp


In [25]:
!g++ -std=c++11 task2.cpp -o task2
!./task2

Минимальное значение: 2
Максимальное значение: 999996
Время выполнения: 2.30433 мс


# Задание 3

    Используя OpenMP, реализуйте параллельный поиск минимального и максимального
    элементов массива из задания 2. Сравните время выполнения последовательной и
    параллельной реализаций.

In [26]:
%%writefile task3.cpp

#include <iostream>   // ввод и вывод данных (cout, endl)
#include <cstdlib>    // rand(), srand()
#include <ctime>      // time() для инициализации генератора случайных чисел
#include <omp.h>      // библиотека OpenMP для параллельных вычислений
#include <chrono>     // измерение времени выполнения

using namespace std;              // чтобы не писать std::
using namespace std::chrono;      // пространство имён для chrono

int main() {
    const int N = 1000000;          // размер массива (1 000 000 элементов)
    const int MAX_VAL = 1000000;    // максимальное значение случайных чисел

    // Динамическое выделение памяти под массив
    int* arr = new int[N];

    // Заполнение массива случайными числами
    // -------------------------------
    srand(time(nullptr));           // инициализация генератора случайных чисел
    for (int i = 0; i < N; ++i) {
        arr[i] = rand() % MAX_VAL + 1;  // числа в диапазоне [1; MAX_VAL]
    }

    // Последовательный поиск минимума и максимума
    // -------------------------------

    // Инициализация min и max первым элементом массива
    int min_seq = arr[0];
    int max_seq = arr[0];

    // Фиксация времени начала последовательного алгоритма
    high_resolution_clock::time_point start_seq =
        high_resolution_clock::now();

    // Проход по массиву и обновление min и max
    for (int i = 1; i < N; ++i) {
        if (arr[i] < min_seq)
            min_seq = arr[i];
        if (arr[i] > max_seq)
            max_seq = arr[i];
    }

    // Фиксация времени окончания последовательного алгоритма
    high_resolution_clock::time_point end_seq =
        high_resolution_clock::now();

    // Вычисление времени выполнения последовательной версии (в мс)
    double seq_time =
        duration<double, milli>(end_seq - start_seq).count();

    // Параллельный поиск минимума и максимума (OpenMP)
    // -------------------------------

    // Начальные значения для параллельного поиска
    int min_par = arr[0];
    int max_par = arr[0];

    // Фиксация времени начала параллельного алгоритма
    high_resolution_clock::time_point start_par =
        high_resolution_clock::now();

    // Параллельный цикл с редукцией по минимуму и максимуму
#pragma omp parallel for reduction(min:min_par) reduction(max:max_par)
    for (int i = 1; i < N; ++i) {
        if (arr[i] < min_par)
            min_par = arr[i];
        if (arr[i] > max_par)
            max_par = arr[i];
    }

    // Фиксация времени окончания параллельного алгоритма
    high_resolution_clock::time_point end_par =
        high_resolution_clock::now();

    // Вычисление времени выполнения параллельной версии (в мс)
    double par_time =
        duration<double, milli>(end_par - start_par).count();

    // Вывод результатов
    cout << "Последовательный минимум: " << min_seq << endl;
    cout << "Последовательный максимум: " << max_seq << endl;
    cout << "Последовательное время: " << seq_time << " мс\n\n";

    cout << "Параллельный минимум: " << min_par << endl;
    cout << "Параллельный максимум: " << max_par << endl;
    cout << "Параллельное время: " << par_time << " мс\n";

    // Освобождение динамически выделенной памяти
    delete[] arr;

    return 0;   // успешное завершение программы
}


Overwriting task3.cpp


In [27]:
! /opt/homebrew/Cellar/gcc/15.2.0/bin/g++-15 -fopenmp task3.cpp -o task3
! ./task3

Последовательный минимум: 5
Последовательный максимум: 999998
Последовательное время: 1.21 мс

Параллельный минимум: 5
Параллельный максимум: 999998
Параллельное время: 0.484 мс


# Задание 4 

    Создайте массив из 5 000 000 чисел и реализуйте вычисление среднего значения
    элементов массива последовательным способом и с использованием OpenMP с редукцией.
    Сравните время выполнения обеих реализаций.

In [28]:
%%writefile task4.cpp

#include <iostream>   // ввод и вывод данных (cout)
#include <cstdlib>    // rand(), srand()
#include <chrono>     // измерение времени выполнения
#include <omp.h>      // поддержка OpenMP

int main() {
    using namespace std;              // упрощаем использование стандартных имён
    using namespace std::chrono;      // пространство имён для chrono

    const size_t N = 5000000; // размер массива (5 000 000 элементов)

    // Динамическое выделение памяти под массив
    int* arr = new int[N];

    // Заполнение массива случайными числами
    // -------------------------------
    srand(time(0));                   // инициализация генератора случайных чисел
    for (size_t i = 0; i < N; i++) {
        arr[i] = rand() % 1000 + 1;   // случайные числа в диапазоне [1; 1000]
    }

    // Последовательное вычисление среднего значения
    // -------------------------------

    // Фиксация времени начала последовательного алгоритма
    auto start_seq = high_resolution_clock::now();

    long long sum_seq = 0;            // сумма элементов массива
    for (size_t i = 0; i < N; i++) {
        sum_seq += arr[i];            // накапливаем сумму
    }

    // Вычисление среднего значения
    double avg_seq = static_cast<double>(sum_seq) / N;

    // Фиксация времени окончания последовательного алгоритма
    auto end_seq = high_resolution_clock::now();

    // Время выполнения последовательной версии (в миллисекундах)
    double time_seq =
        duration<double, milli>(end_seq - start_seq).count();

    // Вывод результата последовательного вычисления
    cout << "Среднее (последовательно): " << avg_seq
         << ", время: " << time_seq << " мс\n";

    // Параллельное вычисление среднего значения (OpenMP)
    // -------------------------------

    long long sum_par = 0;             // общая сумма для параллельного вычисления

    // Фиксация времени начала параллельного алгоритма
    auto start_par = high_resolution_clock::now();

    // Параллельный цикл с редукцией суммы
#pragma omp parallel for reduction(+:sum_par)
    for (size_t i = 0; i < N; i++) {
        sum_par += arr[i];            // каждый поток считает свою часть суммы
    }

    // Вычисление среднего значения для параллельной версии
    double avg_par = static_cast<double>(sum_par) / N;

    // Фиксация времени окончания параллельного алгоритма
    auto end_par = high_resolution_clock::now();

    // Время выполнения параллельной версии (в миллисекундах)
    double time_par =
        duration<double, milli>(end_par - start_par).count();

    // Вывод результата параллельного вычисления
    cout << "Среднее (параллельно): " << avg_par
         << ", время: " << time_par << " мс\n";

    // Освобождение динамически выделенной памяти
    delete[] arr;

    return 0;   // завершение программы
}


Overwriting task4.cpp


In [29]:
! /opt/homebrew/Cellar/gcc/15.2.0/bin/g++-15 -fopenmp task4.cpp -o task4
! ./task4

Среднее (последовательно): 500.587, время: 3.113 мс
Среднее (параллельно): 500.587, время: 0.864 мс
